Dans ce code : Les données AWAY et HOME sont soustraites pour n'avoir que des différences. De ce fait, toutes les variables dans ces modèles sont "des nouvelles variables". Cette technique a été choisie car dans les modèles précédents, les variables les plus importantes ont été celles qui était des différences.
(Variables trop corrélées et gestion des NA grâce a al ligue quelquesoit le code)

In [1]:

import pandas as pd
import numpy as np
import seaborn as sns
#!pip install --upgrade --force-reinstall matplotlib
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import numpy as np
#!pip install tqdm 
#!pip install xgboost
from tqdm import tqdm
#!pip install scikit-learn
#!pip install xgboost
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import lightgbm as lgb
import warnings
import matplotlib.pyplot as plt
#!pip install optuna
import optuna
#!pip install catboost
from catboost import CatBoostClassifier
#!pip install pycaret


In [2]:
train_home_team = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/train_home_team_statistics_df.csv', index_col=0)
train_away_team = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/train_away_team_statistics_df.csv', index_col=0)

train_home_player = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/train_home_player_statistics_df.csv', index_col=0)
train_away_player = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/train_away_player_statistics_df.csv', index_col=0)

train_scores = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/Y_train.csv', index_col=0)

test_home_team = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/test_home_team_statistics_df.csv', index_col=0)
test_away_team = pd.read_csv('C:/Users/agraf/Desktop/uni/master/volume 2 foot (1)/volume 2 foot/test_away_team_statistics_df.csv', index_col=0)

In [3]:
# Drop column: 'TEAM_NAME'
train_home_team = train_home_team.drop(columns=['TEAM_NAME'])
# Pour train_home_team
train_home_team = train_home_team.replace({np.inf: np.nan, -np.inf: np.nan})

# Pour train_away_team
train_away_team = train_away_team.replace({np.inf: np.nan, -np.inf: np.nan})

# Pour test_home_team
test_home_team = test_home_team.replace({np.inf: np.nan, -np.inf: np.nan})

# Pour test_away_team
test_away_team = test_away_team.replace({np.inf: np.nan, -np.inf: np.nan})

#Voir les différentes league :
unique_leagues = train_home_team['LEAGUE'].unique()
print(unique_leagues)

#Calculer la moyenne de chaque colonne par ligue
means_per_league_home = train_home_team.groupby('LEAGUE').transform('mean')

# Remplacer les valeurs NA par les moyennes de la ligue correspondante
train_home_team = train_home_team.fillna(means_per_league_home)

print(train_home_team)

#On enleve "TEAM_INJURIES_5_last_match_std" car elle contient quand meme des NA, ainsi que league car la variable ne nous sert plus
train_home_team = train_home_team.drop(columns=['TEAM_INJURIES_5_last_match_std'])
train_home_team = train_home_team.drop(columns=['LEAGUE'])

# Pour train_away_team
# Drop column: 'TEAM_NAME'
train_away_team = train_away_team.drop(columns=['TEAM_NAME'])

# Calculer la moyenne de chaque colonne par ligue
means_per_league_away = train_away_team.groupby('LEAGUE').transform('mean')

# Remplacer les valeurs NA par les moyennes de la ligue correspondante
train_away_team = train_away_team.fillna(means_per_league_away)

# On enlève "TEAM_INJURIES_5_last_match_std" et 'LEAGUE'
train_away_team = train_away_team.drop(columns=['TEAM_INJURIES_5_last_match_std'])
train_away_team = train_away_team.drop(columns=['LEAGUE'])

# Pour test_home_team

# Remplacer les valeurs NA par les moyennes correspondantes à chaque colonne des équipes home
test_home_team = test_home_team.fillna(means_per_league_home.mean())

# On enlève "TEAM_INJURIES_5_last_match_std"
test_home_team = test_home_team.drop(columns=['TEAM_INJURIES_5_last_match_std'])

# Pour test_away_team

# Remplacer les valeurs NA par les moyennes correspondantes à chaque colonne des équipes away
test_away_team = test_away_team.fillna(means_per_league_away.mean())

# On enlève "TEAM_INJURIES_5_last_match_std"
test_away_team = test_away_team.drop(columns=['TEAM_INJURIES_5_last_match_std'])

['Ligue 1' 'Ligue 2' 'Serie A' 'League One' 'Premier League'
 'Liga Portugal' 'La Liga' 'Superliga' 'Bundesliga' 'Pro League'
 'J-League' 'League Two' 'Eredivisie']
               LEAGUE  TEAM_SHOTS_TOTAL_season_sum  \
ID                                                   
0             Ligue 1                          3.0   
1             Ligue 2                          6.0   
2             Serie A                          4.0   
3          League One                          7.0   
4      Premier League                          3.0   
...               ...                          ...   
12298      League One                          4.0   
12299   Liga Portugal                          4.0   
12300      Bundesliga                          4.0   
12301      League One                          2.0   
12302         La Liga                          2.0   

       TEAM_SHOTS_INSIDEBOX_season_sum  TEAM_SHOTS_OFF_TARGET_season_sum  \
ID                                                      

In [128]:
# Pour les données d'entraînement
# Calculer la différence entre les données d'équipe à domicile et à l'extérieur
train_diff = train_home_team.values - train_away_team.values
train_diff_df = pd.DataFrame(train_diff, index=train_home_team.index, columns=train_home_team.columns.str.replace('HOME_', 'DIFF_'))

# Ajouter la colonne des scores
train_scores = train_scores.loc[train_diff_df.index]
train_diff_df = pd.concat([train_diff_df, train_scores], axis=1)

# Déterminer la variable cible (target_variable)
train_diff_df['target_variable'] = train_diff_df[['HOME_WINS', 'DRAW', 'AWAY_WINS']].idxmax(axis=1)

# Mapper les valeurs cibles en valeurs numériques
target_mapping = {'HOME_WINS': 0, 'DRAW': 1, 'AWAY_WINS': 2}
train_diff_df['target_variable'] = train_diff_df['target_variable'].map(target_mapping)

# Supprimer les colonnes des scores, car elles ne sont plus nécessaires après avoir créé la variable cible
train_diff_df.drop(columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'], inplace=True)

# Pour les données de test
# Calculer la différence entre les données d'équipe à domicile et à l'extérieur
test_diff = test_home_team.values - test_away_team.values
test_diff_df = pd.DataFrame(test_diff, index=test_home_team.index, columns=test_home_team.columns.str.replace('HOME_', 'DIFF_'))




In [129]:
# Définir la variable cible (Y)
target_variable = 'target_variable'

# Calculer la matrice de corrélation sans la variable cible
corr_matrix = train_diff_df.drop(columns=[target_variable]).corr().abs()

# Supprimer les variables ayant une corrélation supérieure à 0.8
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
print(f'Nombre de variables supprimées : {len(to_drop)}')
print(f'Variables supprimées : {to_drop}')

# Supprimer les variables corrélées dans le DataFrame d'origine
train_data2 = train_diff_df.drop(columns=to_drop)
test_data2 = test_diff_df.drop(columns=to_drop, errors='ignore')

Nombre de variables supprimées : 49
Variables supprimées : ['TEAM_SUCCESSFUL_PASSES_season_sum', 'TEAM_SHOTS_TOTAL_season_average', 'TEAM_SHOTS_INSIDEBOX_season_average', 'TEAM_SHOTS_OFF_TARGET_season_average', 'TEAM_SHOTS_ON_TARGET_season_average', 'TEAM_SHOTS_OUTSIDEBOX_season_average', 'TEAM_PASSES_season_average', 'TEAM_SUCCESSFUL_PASSES_season_average', 'TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average', 'TEAM_SAVES_season_average', 'TEAM_CORNERS_season_average', 'TEAM_FOULS_season_average', 'TEAM_YELLOWCARDS_season_average', 'TEAM_REDCARDS_season_average', 'TEAM_OFFSIDES_season_average', 'TEAM_ATTACKS_season_average', 'TEAM_PENALTIES_season_average', 'TEAM_DANGEROUS_ATTACKS_season_average', 'TEAM_GOALS_season_average', 'TEAM_GAME_WON_season_average', 'TEAM_GAME_DRAW_season_average', 'TEAM_GAME_LOST_season_average', 'TEAM_SUCCESSFUL_PASSES_season_std', 'TEAM_REDCARDS_season_std', 'TEAM_SUCCESSFUL_PASSES_5_last_match_sum', 'TEAM_SHOTS_TOTAL_5_last_match_average', 'TEAM_SHOTS_INSIDE

Création des modèles (comme pour l'autre code, l'ordre des modèles dans le code n'est pas l'ordre réel de création)

In [ ]:
# CATBOOST avec optuna pour optimiser les hyper paramètres

import optuna
from sklearn.model_selection import cross_val_score

# Définir les données d'entraînement
target_variable = 'target_variable' 
X_train = train_data2.drop(columns=[target_variable])
y_train = train_data2[target_variable]

# Définir une fonction d'optimisation avec Optuna pour CatBoost
def objective(trial):
    params = {
        'iterations': trial.suggest_categorical('iterations', [150, 300, 500]),
        'depth': trial.suggest_categorical('depth', [4, 8, 10, 12]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'l2_leaf_reg': trial.suggest_categorical('l2_leaf_reg', [5, 9, 12, 14]),
        'border_count': trial.suggest_categorical('border_count', [32, 64, 128]),
        'loss_function': 'MultiClass',
        'random_seed': 64,
        'verbose': 0,
        'early_stopping_rounds': 30
    }
    
    cat_model = CatBoostClassifier(**params)
    scores = cross_val_score(cat_model, X_train, y_train, cv=3, scoring='accuracy')
    return scores.mean()

# Lancer l'optimisation avec Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# Afficher les meilleurs paramètres
print("Meilleurs hyperparamètres trouvés par Optuna :")
print(study.best_params)

# Utiliser les meilleurs paramètres trouvés pour créer le modèle final
best_params = study.best_params
best_cat = CatBoostClassifier(**best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres trouvés
best_cat.fit(X_train, y_train)

# Prédire la variable cible pour la base test_data (les variables explicatives)
y_pred_test = best_cat.predict(test_data2)

# Créer une matrice de zéros pour les prédictions one-hot
y_pred_one_hot = np.zeros((len(y_pred_test), 3), dtype=int)

# Remplir la matrice one-hot en fonction des prédictions
for i, pred in enumerate(y_pred_test):
    y_pred_one_hot[i, pred] = 1

# Créer un DataFrame pour les résultats de prédiction
predictions_one_hot_df = pd.DataFrame(y_pred_one_hot, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])

# Ajouter une colonne ID (utiliser l'index de test_data comme identifiant)
predictions_one_hot_df.insert(0, 'ID', test_data2.index)

# Exporter les résultats dans un fichier CSV
predictions_one_hot_df.to_csv('predictions_catboost_best_one_hot.csv', index=False)

print("Prédictions one-hot enregistrées dans le fichier 'predictions_catboost_best_one_hot.csv'")


In [ ]:
# CATBOOST N2 (avec les hyperparametre d'optuna), dans le but d'obtenir les variables les moins importante dans le modèle

from sklearn.model_selection import cross_val_score

# Définir les données d'entraînement
target_variable = 'target_variable' 
X_train = train_data2.drop(columns=[target_variable])
y_train = train_data2[target_variable]

# Définir les hyperparamètres optimaux trouvés
best_params = {
    'learning_rate': 0.05,
    'l2_leaf_reg': 15,
    'iterations': 50,
    'depth': 6,
    'border_count': 128,
    'loss_function': 'MultiClass',
    'random_seed': 64,
    'verbose': 0
}

# Créer le modèle CatBoost avec les meilleurs hyperparamètres
best_cat = CatBoostClassifier(**best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres trouvés
best_cat.fit(X_train, y_train)

# Afficher les variables qui contribuent le plus au modèle
feature_importances = best_cat.get_feature_importance()
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print("Importance des variables :")
print(feature_importance_df)

# Prédire la variable cible pour la base test_data (les variables explicatives)
y_pred_test = best_cat.predict(test_data2)

# Créer une matrice de zéros pour les prédictions one-hot
y_pred_one_hot = np.zeros((len(y_pred_test), 3), dtype=int)

# Remplir la matrice one-hot en fonction des prédictions
for i, pred in enumerate(y_pred_test):
    y_pred_one_hot[i, pred] = 1

# Créer un DataFrame pour les résultats de prédiction
predictions_one_hot_df = pd.DataFrame(y_pred_one_hot, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])

# Ajouter une colonne ID (utiliser l'index de test_data comme identifiant)
predictions_one_hot_df.insert(0, 'ID', test_data2.index)

# # Exporter les résultats dans un fichier CSV
# predictions_one_hot_df.to_csv('predictions_catboost_best_one_hot.csv', index=False)

# print("Prédictions one-hot enregistrées dans le fichier 'predictions_catboost_best_one_hot.csv'")


Importance des variables :
                                Feature  Importance
21            TEAM_GAME_LOST_season_sum   12.849121
19             TEAM_GAME_WON_season_sum    9.029985
1       TEAM_SHOTS_INSIDEBOX_season_sum    7.367896
22  TEAM_BALL_POSSESSION_season_average    4.700247
3       TEAM_SHOTS_ON_TARGET_season_sum    4.438789
..                                  ...         ...
85        TEAM_ATTACKS_5_last_match_std    0.000000
86  TEAM_SUBSTITUTIONS_5_last_match_std    0.000000
23    TEAM_SUBSTITUTIONS_season_average    0.000000
35      TEAM_BALL_POSSESSION_season_std    0.000000
60  TEAM_SUBSTITUTIONS_5_last_match_sum    0.000000

[90 rows x 2 columns]


In [ ]:
# CATBOOST avec suppression des variables les moins importantes

from sklearn.model_selection import cross_val_score

# Définir les données d'entraînement
target_variable = 'target_variable' 
X_train = train_data2.drop(columns=[target_variable])
y_train = train_data2[target_variable]

# Définir les hyperparamètres optimaux trouvés
best_params = {
    'learning_rate': 0.05,
    'l2_leaf_reg': 15,
    'iterations': 50,
    'depth': 6,
    'border_count': 128,
    'loss_function': 'MultiClass',
    'random_seed': 64,
    'verbose': 0
}

# Créer le modèle CatBoost avec les meilleurs hyperparamètres
best_cat = CatBoostClassifier(**best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres trouvés
best_cat.fit(X_train, y_train)

# Afficher les variables qui contribuent le plus au modèle
feature_importances = best_cat.get_feature_importance()
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Définir un seuil pour l'importance minimale (ici de 1)
importance_threshold = 1 
important_features = feature_importance_df[feature_importance_df['Importance'] > importance_threshold]['Feature']

# Créer un nouveau DataFrame avec uniquement les caractéristiques importantes
X_train_reduced = X_train[important_features]

# Réentraîner le modèle avec les caractéristiques réduites
best_cat.fit(X_train_reduced, y_train)

# Supprimer les mêmes caractéristiques de la base test_data
test_data_reduced = test_data2[important_features]

# Prédire la variable cible pour la base test_data réduite (les variables explicatives)
y_pred_test = best_cat.predict(test_data_reduced)

# Créer une matrice de zéros pour les prédictions one-hot
y_pred_one_hot = np.zeros((len(y_pred_test), 3), dtype=int)

# Remplir la matrice one-hot en fonction des prédictions
for i, pred in enumerate(y_pred_test):
    y_pred_one_hot[i, pred] = 1

# Créer un DataFrame pour les résultats de prédiction
predictions_one_hot_df = pd.DataFrame(y_pred_one_hot, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])

# Ajouter une colonne ID (utiliser l'index de test_data comme identifiant)
predictions_one_hot_df.insert(0, 'ID', test_data2.index)

# Exporter les résultats dans un fichier CSV
predictions_one_hot_df.to_csv('predictions4_catboost_best_one_hot.csv', index=False)

print("Prédictions one-hot enregistrées dans le fichier 'predictions4_catboost_best_one_hot.csv'")


Prédictions one-hot enregistrées dans le fichier 'predictions4_catboost_best_one_hot.csv'


In [ ]:
# CATBOOST avec une création de variables combinées pour essayer d'augmenter l'importance de variables qui ne contribuent
# pas au modèle mais qui le devraient logiquement

from sklearn.model_selection import cross_val_score


# Créer de nouvelles caractéristiques dans test_data2 et train_data2 en multipliant les variables peu utile
train_data2['TEAM_SHOTS_ON_TARGET_5_last_match_std_combined'] = train_data2['TEAM_SHOTS_ON_TARGET_5_last_match_std'] * train_data2['TEAM_DANGEROUS_ATTACKS_5_last_match_std'] * train_data2['TEAM_GOALS_5_last_match_std']
train_data2['TEAM_DANGEROUS_ATTACKS_season_std_combined'] = train_data2['TEAM_DANGEROUS_ATTACKS_season_std'] * train_data2['TEAM_SHOTS_OUTSIDEBOX_season_std']

test_data2['TEAM_SHOTS_ON_TARGET_5_last_match_std_combined'] = test_data2['TEAM_SHOTS_ON_TARGET_5_last_match_std'] * test_data2['TEAM_DANGEROUS_ATTACKS_5_last_match_std'] * test_data2['TEAM_GOALS_5_last_match_std']
test_data2['TEAM_DANGEROUS_ATTACKS_season_std_combined'] = test_data2['TEAM_DANGEROUS_ATTACKS_season_std'] * test_data2['TEAM_SHOTS_OUTSIDEBOX_season_std']

# Supprimer les anciennes colonnes pour éviter les doublons
train_data2 = train_data2.drop(columns=['TEAM_SHOTS_ON_TARGET_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_std', 'TEAM_GOALS_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_season_std', 'TEAM_SHOTS_OUTSIDEBOX_season_std'])
test_data2 = test_data2.drop(columns=['TEAM_SHOTS_ON_TARGET_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_5_last_match_std', 'TEAM_GOALS_5_last_match_std', 'TEAM_DANGEROUS_ATTACKS_season_std', 'TEAM_SHOTS_OUTSIDEBOX_season_std'])

# Définir les données d'entraînement
target_variable = 'target_variable' 
X_train = train_data2.drop(columns=[target_variable])
y_train = train_data2[target_variable]

# Définir les hyperparamètres optimaux trouvés
best_params = {
    'learning_rate': 0.05,
    'l2_leaf_reg': 15,
    'iterations': 50,
    'depth': 6,
    'border_count': 128,
    'loss_function': 'MultiClass',
    'random_seed': 64,
    'verbose': 0
}

# Créer le modèle CatBoost avec les meilleurs hyperparamètres
best_cat = CatBoostClassifier(**best_params)

# Entraîner le modèle avec les meilleurs hyperparamètres trouvés
best_cat.fit(X_train, y_train)

# Prédire la variable cible pour la base test_data (les variables explicatives)
y_pred_test = best_cat.predict(test_data2)

# Créer une matrice de zéros pour les prédictions one-hot
y_pred_one_hot = np.zeros((len(y_pred_test), 3), dtype=int)

# Remplir la matrice one-hot en fonction des prédictions
for i, pred in enumerate(y_pred_test):
    y_pred_one_hot[i, pred] = 1

# Créer un DataFrame pour les résultats de prédiction
predictions_one_hot_df = pd.DataFrame(y_pred_one_hot, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])

# Ajouter une colonne ID (utiliser l'index de test_data comme identifiant)
predictions_one_hot_df.insert(0, 'ID', test_data2.index)

# Exporter les résultats dans un fichier CSV
predictions_one_hot_df.to_csv('predictionscombinee_catboost_best_one_hot.csv', index=False)

print("Prédictions one-hot enregistrées dans le fichier 'predictionscombinee_catboost_best_one_hot.csv'")


In [ ]:
#XGBOOST avec optuna 
target_variable = 'target_variable'  # Remplacer par le nom de ta variable cible
X_train = train_data2.drop(columns=[target_variable])
y_train = train_data2[target_variable]

# Diviser les données d'entraînement en ensemble d'entraînement et de validation
X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(X_train, y_train, test_size=0.2, random_state=64)

# Fonction d'objectif pour l'optimisation d'Optuna
def objective(trial):
    # Définir l'espace de recherche des hyperparamètres
    param = {
        'verbosity': 0,
        'objective': 'multi:softmax',
        'num_class': 3,
        'booster': 'gbtree',
        'eval_metric': 'mlogloss',
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.2),
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 0.01, 0.5)
    }

    # Entraîner le modèle avec les paramètres proposés
    model = xgb.XGBClassifier(**param, random_state=64)
    model.fit(X_train_split, y_train_split)

    # Prédire sur l'ensemble de validation
    y_pred_valid = model.predict(X_valid_split)

    # Calculer la précision sur l'ensemble de validation
    accuracy = accuracy_score(y_valid_split, y_pred_valid)
    
    # Retourner la précision (plus la valeur est élevée, mieux c'est)
    return accuracy

# Créer une étude Optuna et trouver les meilleurs hyperparamètres
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=-1)

# Afficher les meilleurs hyperparamètres trouvés
best_params = study.best_params
print("Meilleurs hyperparamètres trouvés :")
print(best_params)

# Utiliser les meilleurs hyperparamètres trouvés pour entraîner le modèle final
best_xgb = xgb.XGBClassifier(**best_params, objective='multi:softmax', num_class=3, random_state=64)
best_xgb.fit(X_train, y_train)

# Prédire la variable cible pour la base test_data (les variables explicatives)
y_pred_test = best_xgb.predict(test_data2)

# Créer une matrice de zéros pour les prédictions one-hot
y_pred_one_hot = np.zeros((len(y_pred_test), 3), dtype=int)

# Remplir la matrice one-hot en fonction des prédictions
for i, pred in enumerate(y_pred_test):
    y_pred_one_hot[i, pred] = 1

# Créer un DataFrame pour les résultats de prédiction
predictions_one_hot_df = pd.DataFrame(y_pred_one_hot, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])

# Ajouter une colonne ID (utiliser l'index de test_data comme identifiant)
predictions_one_hot_df.insert(0, 'ID', test_data2.index)

# Exporter les résultats dans un fichier CSV
predictions_one_hot_df.to_csv('predictions2_xgboost_optuna_one_hot.csv', index=False)

print("Prédictions one-hot enregistrées dans le fichier 'predictions2_xgboost_optuna_one_hot.csv'")


In [ ]:
# Random forest 

seed = 64

target_variable = 'target_variable'  
X_train = train_data2.drop(columns=[target_variable])
y_train = train_data2[target_variable]

rf = RandomForestClassifier(random_state=seed)

param_distributions = {
    'n_estimators': [50, 100, 200],  
    'max_features': ['sqrt', 'log2'], 
    'max_depth': [5, 8, 10],  
    'min_samples_split': [10, 15, 20],  
    'min_samples_leaf': [4, 6, 8], 
    'bootstrap': [True]
}

# Utiliser RandomizedSearchCV pour trouver les meilleurs hyperparamètres
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_distributions,
                               n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Entraîner à nouveau
rf_random.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print("Meilleurs hyperparamètres trouvés :")
print(rf_random.best_params_)

# Prédire sur les données d'entraînement avec les meilleurs hyperparamètres trouvés
best_rf = rf_random.best_estimator_
y_pred_train = best_rf.predict(X_train)

# Calculer et afficher la précision sur les données d'entraînement
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f'Précision sur les données d\'entraînement après ajustement : {accuracy_train:.4f}')


In [ ]:
# Random forest avec d'autres hyper paramètres pour tester

seed = 64

target_variable = 'target_variable'  # Remplace par le nom de ta variable cible
X_train = train_data2.drop(columns=[target_variable])
y_train = train_data2[target_variable]

rf = RandomForestClassifier(random_state=seed)

param_distributions = {
    'n_estimators': [50, 100, 150, 125, 75],
    'max_features': ['sqrt', 'log2', 0.4, 0.3, 0.5, 0.2],  
    'max_depth': [7, 8, 9, 10, 11, 12, 15, 20], 
    'min_samples_split': [8, 9, 10, 12, 14, 11, 13], 
    'min_samples_leaf': [4, 5, 6, 7, 8, 9, 10],  
    'bootstrap': [True]
}

# Utiliser RandomizedSearchCV pour trouver les meilleurs hyperparamètres
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_distributions,
                               n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Entraîner à nouveau
rf_random.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print("Meilleurs hyperparamètres trouvés :")
print(rf_random.best_params_)

# Prédire sur les données d'entraînement avec les meilleurs hyperparamètres trouvés
best_rf2 = rf_random.best_estimator_
y_pred_train = best_rf2.predict(X_train)

# Calculer et afficher la précision sur les données d'entraînement
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f'Précision sur les données d\'entraînement après ajustement : {accuracy_train:.4f}')


Validation croisée pour comparer les modèles entre eux

In [ ]:
# Effectuer une validation croisée sur le modèle optimisé
cv_scores = cross_val_score(best_rf, X_train, y_train, cv=5, scoring='accuracy')

# Calculer la précision moyenne et l'écart-type
cv_mean = np.mean(cv_scores)
cv_std = np.std(cv_scores)

print(f'Précision moyenne avec validation croisée à 5 folds : {cv_mean:.4f}')
print(f'Écart-type des précisions : {cv_std:.4f}')

In [ ]:
# Accéder à l'importance des variables dans le modèle entraîné
feature_importances = best_rf2.feature_importances_

# Créer un DataFrame pour associer les importances aux noms des caractéristiques
features = X_train.columns 
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

# Trier les variables par ordre décroissant d'importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Afficher les variables les plus importantes
print(importance_df)

# Visualiser l'importance des variables avec un graphique en barres
plt.figure(figsize=(10, 8))
plt.barh(importance_df['Feature'][:15], importance_df['Importance'][:15], color='skyblue')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Top 15 Variables les Plus Importantes')
plt.gca().invert_yaxis()
plt.show()

Test de random forest en mettant un seuil d'importance des variables

In [ ]:
# Définir un seuil pour l'importance des variables
importance_threshold = 0.00955

# Identifier les variables importantes
important_features = importance_df[importance_df['Importance'] > importance_threshold]['Feature'].tolist()

# Afficher les variables à conserver
print("Variables conservées :", important_features)

In [ ]:
# Mettre à jour les jeux de données avec seulement les variables importantes
X_train_filtered = X_train[important_features]
test_data_filtered = test_data2[important_features]

# Afficher la nouvelle forme des jeux de données
print("Nouveau nombre de variables :", X_train_filtered.shape[1])

In [ ]:


seed = 64

# Définir les données d'entraînement avec les variables restantes
target_variable = 'target_variable'  
X_train_filtered = X_train[important_features]  # Utiliser uniquement les variables restantes
y_train = train_data2[target_variable]

rf = RandomForestClassifier(random_state=seed)

param_distributions = {
    'n_estimators': [50, 100, 150, 125, 75],  
    'max_features': ['sqrt', 'log2', 0.4, 0.3, 0.5, 0.2],  
    'max_depth': [7, 8, 9, 10, 11, 12, 15, 20],  
    'min_samples_split': [8, 9, 10, 12, 14, 11, 13],  
    'min_samples_leaf': [4, 5, 6, 7, 8, 9, 10], 
    'bootstrap': [True]
}

# Utiliser RandomizedSearchCV pour trouver les meilleurs hyperparamètres
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_distributions,
                               n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=-1)

# Entraîner à nouveau
rf_random.fit(X_train_filtered, y_train)

# Afficher les meilleurs hyperparamètres
print("Meilleurs hyperparamètres trouvés :")
print(rf_random.best_params_)

# Prédire sur les données d'entraînement avec les meilleurs hyperparamètres trouvés
best_rf2 = rf_random.best_estimator_
y_pred_train = best_rf2.predict(X_train_filtered)

# Prédire la variable cible pour la base test_data (les variables explicatives)
y_pred_test = best_rf2.predict(test_data2[important_features])

# Créer une matrice de zéros pour les prédictions one-hot
y_pred_one_hot = np.zeros((len(y_pred_test), 3), dtype=int)

# Remplir la matrice one-hot en fonction des prédictions
for i, pred in enumerate(y_pred_test):
    y_pred_one_hot[i, pred] = 1

# Créer un DataFrame pour les résultats de prédiction
predictions_one_hot_df = pd.DataFrame(y_pred_one_hot, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])

# Ajouter une colonne ID (utiliser l'index de test_data comme identifiant)
predictions_one_hot_df.insert(0, 'ID', test_data2.index)

# Exporter les résultats dans un fichier CSV
predictions_one_hot_df.to_csv('predictions2_rf_best_one_hot.csv', index=False)

print("Prédictions one-hot enregistrées dans le fichier 'predictions2_rf_best_one_hot.csv'")


In [ ]:
y_pred_test = best_rf2.predict(test_data2)

# Créer un DataFrame pour les résultats de prédiction
# On suppose que test_data a une colonne d'ID ou d'index pour identifier chaque prédiction
predictions = pd.DataFrame({
    'ID': test_data2.index,  
    'target_variable': y_pred_test
})

# Exporter les résultats dans un fichier CSV
predictions.to_csv('predictions2.csv', index=False)

print("Prédictions enregistrées dans le fichier 'predictions2.csv'")

# Charger les prédictions depuis le fichier CSV 'y_pred_test'
y_pred_df = pd.read_csv('predictions2.csv')

# Assumer que le fichier CSV contient une colonne 'target_variable' qui a les valeurs prédictes (0, 1, 2)
y_pred = y_pred_df['target_variable']

# Créer une matrice de zéros pour les prédictions one-hot
# Chaque ligne représente un match, chaque colonne représente une classe : [HOME_WINS, DRAW, AWAY_WINS]
y_pred_one_hot = np.zeros((len(y_pred), 3), dtype=int)

# Remplir la matrice one-hot en fonction des prédictions
for i, pred in enumerate(y_pred):
    y_pred_one_hot[i, pred] = 1

# Créer un DataFrame pour les résultats de prédiction
predictions2_one_hot_df = pd.DataFrame(y_pred_one_hot, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])

# Ajouter une colonne ID (si nécessaire, utiliser l'index du fichier initial)
if 'ID' in y_pred_df.columns:
    predictions2_one_hot_df.insert(0, 'ID', y_pred_df['ID'])
else:
    predictions2_one_hot_df.insert(0, 'ID', y_pred_df.index)

# Exporter les résultats dans un nouveau fichier CSV
predictions2_one_hot_df.to_csv('predictions2_one_hot_df.csv', index=False)

print("Prédictions one-hot enregistrées dans le fichier 'predictions2_one_hot_df.csv'")

In [ ]:
# Test d'un light GBM

# Définir une valeur de seed pour la reproductibilité
seed = 64

# Charger et entraîner le modèle LightGBM
model = lgb.LGBMClassifier(objective='multiclass', num_class=3, random_state=64)
model.fit(X_train, y_train)  # Entraîner le modèle sur les données d'entraînement

# Prédire la variable cible pour la base test_data (les variables explicatives)
y_pred_test = model.predict(test_data2)

# Créer une matrice de zéros pour les prédictions one-hot
# Chaque ligne représente un match, chaque colonne représente une classe : [HOME_WINS, DRAW, AWAY_WINS]
y_pred_one_hot = np.zeros((len(y_pred_test), 3), dtype=int)

# Remplir la matrice one-hot en fonction des prédictions
for i, pred in enumerate(y_pred_test):
    y_pred_one_hot[i, pred] = 1

# Créer un DataFrame pour les résultats de prédiction
predictions3_one_hot_df = pd.DataFrame(y_pred_one_hot, columns=['HOME_WINS', 'DRAW', 'AWAY_WINS'])

# Ajouter une colonne ID (si nécessaire, utiliser l'index du fichier test_data comme identifiant)
predictions3_one_hot_df.insert(0, 'ID', test_data2.index)

# Exporter les résultats dans un nouveau fichier CSV avec des entiers
predictions3_one_hot_df.to_csv('predictions2_lightgbm_one_hot.csv', index=False)

print("Prédictions one-hot enregistrées dans le fichier 'predictions2_lightgbm_one_hot.csv'")